In [ ]:
!pip install transformers==4.16.0

In [17]:
from transformers import AutoTokenizer, AutoConfig, EncoderDecoderModel
import torch
from torch import nn
import torch.nn.functional as F
from math import sqrt

# Transformer Decoder

Come abbiamo visto nella sezione di teoria, il blocco Decoder calcola l'attention in maniera similare al blocco Encoder, tuttavia, l'attenzione non viene messa su tutti i token di input, ma solamente su quelli non mascherati, ovvero quelli antecedenti alla parola/token che stiamo considerando

Ricalcoliamo quindi i nostri vettori di query, key e value

In [18]:
model_ckpt = "patrickvonplaten/bert2bert_cnn_daily_mail"
text = "time flies like an arrow"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = EncoderDecoderModel.from_pretrained(model_ckpt)

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [19]:
model

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [20]:
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [23]:
config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.decoder.hidden_size)
token_emb

Embedding(30522, 768)

In [24]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()

torch.Size([1, 5, 768])

Definiamo una matrice di maschera che rende visibili solamente i token di input antecedenti a quello che stiamo considerando! La maschera assume quindi la forma di una matrice triangolare inferiore

In [25]:
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [26]:
mask

tensor([[[1., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1.]]])

Calcoliamo gli scores come fatto per il blocco Encoder

In [27]:
query = key = value = inputs_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
scores.size()

torch.Size([1, 5, 5])

Mascheriamo gli scores utilizzando la nostra matrice di maschera

In [28]:
scores.masked_fill(mask == 0, -float("inf"))

tensor([[[29.3284,    -inf,    -inf,    -inf,    -inf],
         [ 1.5654, 27.5266,    -inf,    -inf,    -inf],
         [ 0.2496, -0.0899, 29.4625,    -inf,    -inf],
         [ 0.0464, -0.1138, -0.1201, 27.7881,    -inf],
         [-0.6187,  0.5058,  1.3404, -1.0054, 30.3363]]],
       grad_fn=<MaskedFillBackward0>)

A questo punto, come fatto in precedenza, ci calcoliamo i nostri valori di Attention andando a passare gli scores in un layer di softmax e poi moltiplicando il risultato con la matrice dei values

In [29]:
weights = F.softmax(scores, dim=-1)
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

In [30]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

Riepilogando, creiamo una funzione che calcola i valori di Self-Masked Attention partendo dalle 3 matrici di query, key e value:

In [31]:
def scaled_dot_product_attention(query, key, value, mask=None):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float("-inf"))
    weights = F.softmax(scores, dim=-1)
    return weights.bmm(value)

Anche per le architetture Decoder possiamo quindi definire i moduli di Attention e MultiHead Attention, necessari per costruire un blocco Decoder

In [32]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state, mask=None):
        attn_outputs = scaled_dot_product_attention(
            self.q(hidden_state), self.k(hidden_state), self.v(hidden_state), mask=mask)
        return attn_outputs

In [33]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config, mask=None):
        super().__init__()
        embed_dim = config.decoder.hidden_size
        num_heads = config.decoder.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state, mask=None):
        x = torch.cat([h(hidden_state, mask) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x

In [34]:
multihead_attn = MultiHeadAttention(config, mask=mask)
attn_output = multihead_attn(inputs_embeds)
attn_output.size()

torch.Size([1, 5, 768])

Definiamo il modulo FeedForward

In [36]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.decoder.hidden_size, config.decoder.intermediate_size)
        self.linear_2 = nn.Linear(config.decoder.intermediate_size, config.decoder.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.decoder.hidden_dropout_prob)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

A questo punto possiamo definire il layer di TransformerDecoder

In [37]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, config, mask=None):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.decoder.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.decoder.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x, mask=None):
        # Apply layer normalization and then copy input into query, key, value
        hidden_state = self.layer_norm_1(x)
        # Apply attention with a skip connection
        x = x + self.attention(hidden_state, mask)
        # Apply feed-forward layer with a skip connection
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

In [45]:
class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size,
                                             config.decoder.hidden_size)
        self.position_embeddings = nn.Embedding(config.decoder.max_position_embeddings,
                                                config.decoder.hidden_size)
        self.layer_norm = nn.LayerNorm(config.decoder.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        # Create position IDs for input sequence
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
        # Create token and position embeddings
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        # Combine token and position embeddings
        embeddings = token_embeddings + position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [48]:
class TransformerDecoder(nn.Module):
    def __init__(self, config, mask=None):
        super().__init__()
        self.embeddings = Embeddings(config)
        self.layers = nn.ModuleList([TransformerDecoderLayer(config) for _ in range(config.decoder.num_hidden_layers)])

    def forward(self, x, mask):
        x = self.embeddings(x)
        x = self.layers[0](x, mask)
        for layer in self.layers[1:]:
            x = layer(x)
        return x

In [49]:
decoder = TransformerDecoder(config)
decoder(inputs.input_ids, mask).size()

torch.Size([1, 5, 768])

In [50]:
decoder(inputs.input_ids, mask)

tensor([[[-0.8775, -2.0814,  2.0657,  ...,  0.1789,  0.3469,  1.0051],
         [-1.9024, -2.3523,  0.6836,  ..., -1.6603, -0.7521, -0.4312],
         [-1.2800, -0.5740, -4.6542,  ...,  1.2421, -0.7086,  1.7556],
         [-3.1928, -3.1439,  3.4209,  ..., -0.0666, -2.0549,  1.4300],
         [-1.0770, -1.9408, -2.0219,  ..., -0.3106, -7.5593,  1.3381]]],
       grad_fn=<AddBackward0>)

In [51]:
decoder

TransformerDecoder(
  (embeddings): Embeddings(
    (token_embeddings): Embedding(30522, 768)
    (position_embeddings): Embedding(512, 768)
    (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (layers): ModuleList(
    (0-11): 12 x TransformerDecoderLayer(
      (layer_norm_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layer_norm_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attention): MultiHeadAttention(
        (heads): ModuleList(
          (0-11): 12 x AttentionHead(
            (q): Linear(in_features=768, out_features=64, bias=True)
            (k): Linear(in_features=768, out_features=64, bias=True)
            (v): Linear(in_features=768, out_features=64, bias=True)
          )
        )
        (output_linear): Linear(in_features=768, out_features=768, bias=True)
      )
      (feed_forward): FeedForward(
        (linear_1): Linear(in_features=768, out_features=3072, b